# Training Status Process
> Process to handle training data stream

In [ ]:
# | default_exp training_status_process

In [ ]:
# | export

import random
import traceback
from datetime import datetime, timedelta
from os import environ
from time import sleep
from typing import *

import asyncio
import numpy as np
import pandas as pd
from asyncer import asyncify, create_task_group
from contextlib import contextmanager
from fastapi import FastAPI
from fastcore.meta import delegates
from fast_kafka_api.application import FastKafkaAPI
from sqlalchemy.exc import NoResultFound
from sqlalchemy import create_engine as sqlalchemy_create_engine
from sqlalchemy.engine import Engine
from sqlmodel import Session, select, func

import airt_service
from airt_service.users import User
from airt_service.data.clickhouse import get_count_for_account_ids
from airt_service.db.models import (
    create_connection_string,
    get_db_params_from_env_vars,
    get_engine,
    get_session_with_context,
    User,
    TrainingStreamStatus,
)
from airt.logger import get_logger
from airt.patching import patch

23-02-07 10:28:47.310 [INFO] airt.executor.subcommand: Module loaded.


In [ ]:
import json
import threading
from pathlib import Path

import pytest
import uvicorn
from confluent_kafka import Producer, Consumer
from _pytest.monkeypatch import MonkeyPatch

from airt_service.confluent import confluent_kafka_config, create_topics_for_user
from airt_service.db.models import create_user_for_testing
from airt_service.helpers import set_env_variable_context
from airt_service.server import create_ws_server
from airt_service.sanitizer import sanitized_print
from airt_service.uvicorn_helpers import run_uvicorn

In [ ]:
test_username = create_user_for_testing()
display(test_username)

'jbpwshcvio'

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
# | export


@patch(cls_method=True)
def _create(
    cls: TrainingStreamStatus,
    *,
    account_id: int,
    application_id: Optional[str] = None,
    model_id: str,
    model_type: str,
    event: str,
    count: int,
    total: int,
    user: User,
) -> TrainingStreamStatus:
    """
    Method to create event

    Args:
        account_id: account id
        application_id: Id of the application in case there is more than one for the AccountId
        model_id: User supplied ID of the model trained
        model_type: Model type
        event: one of start, upload, end
        count: current count of rows in clickhouse db
        total: total no. of rows sent by user
        user: user object
        session: session object

    Returns:
        created object of type TrainingStreamStatus
    """
    with get_session_with_context() as session:
        training_event = TrainingStreamStatus(
            account_id=account_id,
            application_id=application_id,
            model_id=model_id,
            model_type=model_type,
            event=event,
            count=count,
            total=total,
            user_id=user.id,
        )
        session.add(training_event)
        session.commit()
        session.refresh(training_event)

    return training_event

In [ ]:
throwaway_username = create_user_for_testing()
with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == throwaway_username)).one()

    test_end_event = TrainingStreamStatus._create(
        account_id=789,
        model_id="ChurnModelForDrivers",
        model_type="churn",
        event="end",
        count=0,
        total=1000,
        user=user,
    )
    display(test_end_event)
    assert test_end_event.id
    assert test_end_event.event == "end"

TrainingStreamStatus(event=<TrainingEvent.end: 'end'>, id=14, account_id=789, model_id='ChurnModelForDrivers', count=0, created=datetime.datetime(2023, 2, 7, 10, 28, 48), uuid=UUID('b81e55c3-7c0c-4046-b9ee-28c478308a3e'), application_id=None, model_type='churn', total=1000, user_id=13)

In [ ]:
# with get_session_with_context() as session:
#     user = session.exec(select(User).where(User.username == throwaway_username)).one()

#     TrainingStreamStatus._create(
#         account_id=112, event="start", count=0, total=1000, user=user, session=session
#     )
#     TrainingStreamStatus._create(
#         account_id=112, event="upload", count=500, total=1000, user=user, session=session
#     )
#     TrainingStreamStatus._create(
#         account_id=112, event="end", count=1000, total=1000, user=user, session=session
#     )

#     TrainingStreamStatus._create(
#         account_id=141, event="start", count=0, total=3000, user=user, session=session
#     )
#     TrainingStreamStatus._create(
#         account_id=141, event="upload", count=100, total=3000, user=user, session=session
#     )

#     TrainingStreamStatus._create(
#         account_id=600, event="start", count=0, total=6000, user=user, session=session
#     )

In [ ]:
# | export


@contextmanager
@delegates(sqlalchemy_create_engine)
def create_sqlalchemy_engine(
    url: str, **kwargs: Dict[str, Any]
) -> Generator[Engine, None, None]:
    sqlalchemy_engine = sqlalchemy_create_engine(url, **kwargs)
    try:
        yield sqlalchemy_engine
    finally:
        sqlalchemy_engine.dispose()


def get_recent_event_for_user(user: User) -> pd.DataFrame:
    """
    Get recent event for user

    Args:
        user: user object to get recent events

    Returns:
        A list of recent events for given user
    """
    conn_str = create_connection_string(**get_db_params_from_env_vars())  # type: ignore

    with create_sqlalchemy_engine(conn_str) as engine:
        # Get all rows from table
        df = pd.read_sql_table(table_name="trainingstreamstatus", con=engine)

    # Filter events for given user and group by account_id
    events_for_user = (
        df.loc[df["user_id"] == user.id]
        .sort_values("id", ascending=False)
        .groupby(
            by=["account_id", "application_id", "model_id"],
            as_index=False,
            dropna=False,
        )
        .first()
    )
    
    events_for_user = events_for_user.rename(
        columns={"count": "prev_count", "account_id": "AccountId"}
    )
    
    events_for_user = events_for_user.set_index("AccountId")

    # Leave 'end' events
    events_for_user = events_for_user.loc[events_for_user["event"] != "end"].sort_values(
        "AccountId", ascending=True
    )
    
    return events_for_user

In [ ]:
end_count = 1_000_000

assert not "actual" in dir(), "restart the kernel please before running this again"

with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()
    actual = get_recent_event_for_user(user=user)
    assert actual.empty, actual
    test_start_event = TrainingStreamStatus._create(
        account_id=999,
        model_id="ChurnModelForDrivers",
        model_type="churn",
        event="start",
        count=0,
        total=10000,
        user=user,
    )
    test_end_event = TrainingStreamStatus._create(
        account_id=999,
        model_id="ChurnModelForDrivers",
        model_type="churn",
        event="end",
        count=10000,
        total=10000,
        user=user,
    )

    test_start_event = TrainingStreamStatus._create(
        account_id=666,
        model_id="ChurnModelForDrivers",
        model_type="churn",
        event="start",
        count=0,
        total=1000,
        user=user,
    )

    actual = get_recent_event_for_user(user=user)
    display(actual)
    assert len(actual) == 1
    actual.iloc[0]["event"] == test_start_event.event
    #     actual.iloc[0]["user_id"] == test_start_event.user.id
    actual.index[0] == test_start_event.account_id
    test_recent_events_df = actual.copy()

,application_id,model_id,event,id,uuid,prev_count,total,created,user_id,model_type
AccountId,,,,,,,,,,
666,NaN,ChurnModelForDrivers,start,17,4b9c3a305eab4e57a9de4fecf6a0d08d,0,1000,2023-02-07 10:28:48,12,churn


In [ ]:
# | export


def get_count_from_training_data_ch_table(
    account_ids: List[Union[int, str]]
) -> pd.DataFrame:
    """
    Get count of all rows for given account ids from clickhouse table

    Args:
        account_ids: List of account_ids to get count

    Returns:
        Count for the given account id
    """
    return airt_service.data.clickhouse.get_count_for_account_ids(
        account_ids=account_ids,
        username=environ["KAFKA_CH_USERNAME"],
        password=environ["KAFKA_CH_PASSWORD"],
        host=environ["KAFKA_CH_HOST"],
        port=int(environ["KAFKA_CH_PORT"]),
        database=environ["KAFKA_CH_DATABASE"],
        table=environ["KAFKA_CH_TABLE"],
        protocol=environ["KAFKA_CH_PROTOCOL"],
    )

In [ ]:
with MonkeyPatch.context() as monkeypatch:
    monkeypatch.setattr(
        "__main__.get_count_from_training_data_ch_table",
        lambda account_ids: pd.DataFrame(
            {
                "curr_count": [999] * len(account_ids),
                "AccountId": account_ids,
                "curr_check_on": [datetime.utcnow()] * len(account_ids),
            }
        ).set_index("AccountId"),
    )
    actual = get_count_from_training_data_ch_table(account_ids=[500])
    display(actual)
    assert actual.iloc[0]["curr_count"] == 999, actual

In [ ]:
# | export


def get_user(username: str) -> User:
    """Get the user object for the given username

    Args:
        username: Username as a string

    Returns:
        The user object
    """
    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == username)).one()

    return user

In [ ]:
with get_session_with_context() as session:
    actual = get_user(username=test_username)
    assert actual.username == test_username

In [ ]:
# | export


async def process_row(
    row: pd.Series,
    user: User,
    fast_kafka_api_app: FastKafkaAPI,
):
    """
    Process a single row, update mysql db and send status message to kafka

    Args:
        row: pandas row
        user: user object
    """
    if not row["action"]:
        return

    async_training_stream_status_create = asyncify(TrainingStreamStatus._create)

    account_id = row.name
    application_id = None if np.isnan(row["application_id"]) else row["application_id"]

    upload_event = await async_training_stream_status_create(  # type: ignore
        account_id=account_id,
        application_id=application_id,
        model_id=row["model_id"],
        model_type=row["model_type"],
        event=row["action"],
        count=row["curr_count"],
        total=row["total"],
        user=user,
    )
    await fast_kafka_api_app.to_infobip_training_data_status(
        account_id=account_id,
        application_id=application_id,
        model_id=row["model_id"],
        no_of_records=row["curr_count"],
        total_no_of_records=row["total"],
    )

In [ ]:
display(test_recent_events_df)
test_ch_df = pd.DataFrame(
    {
        "curr_count": [500],
        "AccountId": 666,
        "curr_check_on": [datetime.utcnow()],
    }
).set_index("AccountId")
display(test_ch_df)

merged = pd.merge(test_recent_events_df, test_ch_df, on="AccountId")
merged["action"] = np.where(
    merged["curr_count"] != merged["prev_count"],
    "upload",
    np.where(
        merged["curr_check_on"] - merged["created"] > pd.Timedelta(seconds=10),
        "end",
        None,
    ),
)
display(merged)

dummy_fast_kafka_api = FastKafkaAPI(FastAPI())


async def dummy_to_infobip_training_data_status(*args, **kwargs):
    logger.info("from dummy func for to_infobip_training_data_status")


dummy_fast_kafka_api.to_infobip_training_data_status = (
    dummy_to_infobip_training_data_status
)

with get_session_with_context() as session:
    user = session.exec(select(User).where(User.username == test_username)).one()

    for index, row in merged.iterrows():

        await process_row(row, user=user, fast_kafka_api_app=dummy_fast_kafka_api)

with get_session_with_context() as session:
    most_recent_event = session.exec(
        select(TrainingStreamStatus)
        .where(TrainingStreamStatus.user == user)
        #         .where(TrainingStreamStatus.account_id == 666)
        .order_by(TrainingStreamStatus.id.desc())
        .limit(1)
    ).one()
    display(f"{most_recent_event=}")
    assert most_recent_event.account_id == 666
    assert most_recent_event.event == "upload"

In [ ]:
# | export


async def process_dataframes(
    recent_events_df: pd.DataFrame,
    ch_df: pd.DataFrame,
    *,
    user: User,
    end_timedelta: int = 30,
    fast_kafka_api_app: FastKafkaAPI,
):
    """
    Process mysql, clickhouse dataframes and take action if needed

    Args:
        recent_events_df: recent events as pandas dataframe from mysql db
        ch_df: count from clickhouse table as dataframe
        user: user object
        end_timedelta: timedelta in seconds to use to determine whether upload is over or not
    """
    df = pd.merge(recent_events_df, ch_df, on="AccountId")
    xs = np.where(  # type: ignore
        df["curr_check_on"].subtract(df["created"])
        > pd.Timedelta(seconds=end_timedelta),
        "end",
        None,
    )
    df["action"] = np.where(
        df["curr_count"] != df["prev_count"],
        "upload",
        xs,
    )

    async with create_task_group() as task_group:
        for account_id, row in df.iterrows():
            task_group.soonify(process_row)(
                row=row, user=user, fast_kafka_api_app=fast_kafka_api_app
            )

In [ ]:
dummy_fast_kafka_api = FastKafkaAPI(FastAPI())


async def dummy_to_infobip_training_data_status(*args, **kwargs):
    logger.info("from dummy func for to_infobip_training_data_status")


dummy_fast_kafka_api.to_infobip_training_data_status = (
    dummy_to_infobip_training_data_status
)


with get_session_with_context() as session:
    test_upload_event = TrainingStreamStatus._create(
        account_id=666,
        model_id="ChurnModelForDrivers",
        model_type="churn",
        event="upload",
        count=1000,
        total=1000,
        user=user,
    )

    user = session.exec(select(User).where(User.username == test_username)).one()
    test_recent_events = get_recent_event_for_user(user=user)
    display(test_recent_events)
    test_ch_df = pd.DataFrame(
        {"curr_count": [1000], "AccountId": [666], "curr_check_on": [datetime.utcnow()]}
    ).set_index("AccountId")

    await process_dataframes(
        recent_events_df=test_recent_events,
        ch_df=test_ch_df,
        user=user,
        fast_kafka_api_app=dummy_fast_kafka_api,
    )
    changed_recent_events = get_recent_event_for_user(user=user)
    pd.testing.assert_frame_equal(test_recent_events, changed_recent_events)
    #     assert test_recent_events == changed_recent_events

    sleep(12)
    test_ch_df = pd.DataFrame(
        {"curr_count": [1000], "AccountId": [666], "curr_check_on": [datetime.utcnow()]}
    ).set_index("AccountId")
    await process_dataframes(
        recent_events_df=test_recent_events,
        ch_df=test_ch_df,
        user=user,
        end_timedelta=10,
        fast_kafka_api_app=dummy_fast_kafka_api,
    )

    changed_recent_events = get_recent_event_for_user(user=user)
    display(changed_recent_events)
    assert changed_recent_events.empty

with get_session_with_context() as session:
    most_recent_event = session.exec(
        select(TrainingStreamStatus)
        .where(TrainingStreamStatus.user == user)
        #         .where(TrainingStreamStatus.account_id == 666)
        .order_by(TrainingStreamStatus.id.desc())
        .limit(1)
    ).one()
    display(f"{most_recent_event=}")
    assert most_recent_event.account_id == 666
    assert most_recent_event.event == "end"

In [ ]:
# | export


async def process_training_status(username: str, fast_kafka_api_app: FastKafkaAPI):
    """
    An infinite loop to keep track of training_data uploads from user

    Args:
        username: username of user to track training data uploads
    """
    async_get_user = asyncify(get_user)
    async_get_recent_event_for_user = asyncify(get_recent_event_for_user)
    async_get_count_from_training_data_ch_table = asyncify(
        get_count_from_training_data_ch_table
    )

    while True:
        #         logger.info(f"Starting the process loop")
        try:
            user = await async_get_user(username)
            recent_events_df = await async_get_recent_event_for_user(user=user)
            if not recent_events_df.empty:
                ch_df = await async_get_count_from_training_data_ch_table(
                    account_ids=recent_events_df.index.tolist()
                )
                await process_dataframes(
                    recent_events_df=recent_events_df,
                    ch_df=ch_df,
                    user=user,  # type: ignore
                    fast_kafka_api_app=fast_kafka_api_app,
                )
        except Exception as e:
            logger.info(
                f"Error in process_training_status - {e}, {traceback.format_exc()}"
            )

        await asyncio.sleep(random.randint(5, 20))  # nosec B311

In [ ]:
definitions = [
    "appLaunch",
    "sign_in",
    "sign_out",
    "add_to_cart",
    "purchase",
    "custom_event_1",
    "custom_event_2",
    "custom_event_3",
]


applications = ["DriverApp", "PUBG", "COD"]


def generate_n_rows_for_training_data(n: int, seed: int = 42):
    rng = np.random.default_rng(seed=seed)
    #     account_id = rng.choice([4000, 5000, 500], size=n)
    account_id = 6000
    definition_id = rng.choice(definitions, size=n)
    application = rng.choice(applications, size=n)
    occurred_time_ticks = rng.integers(
        datetime(year=2022, month=1, day=1).timestamp() * 1000,
        datetime(year=2022, month=11, day=1).timestamp() * 1000,
        size=n,
    )
    occurred_time = pd.to_datetime(occurred_time_ticks, unit="ms").strftime(
        "%Y-%m-%dT%H:%M:%S.%f"
    )
    person_id = rng.integers(n // 10, size=n)

    df = pd.DataFrame(
        {
            "AccountId": account_id,
            "Application": application,
            "DefinitionId": definition_id,
            "OccurredTimeTicks": occurred_time_ticks,
            "OccurredTime": occurred_time,
            "PersonId": person_id,
        }
    )
    return json.loads(df.to_json(orient="records"))


generate_n_rows_for_training_data(100)[-1]

In [ ]:
def delivery_report(err, msg):
    """Called once for each message produced to indicate delivery result.
    Triggered by poll() or flush()."""
    if err is not None:
        sanitized_print("Message delivery failed: {}".format(err))
    else:
        #         sanitized_print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))
        pass

In [ ]:
def test_process_training_status():
    logger.info("I am done at tests")
    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == test_username)).one()
        test_start_event = TrainingStreamStatus._create(
            account_id=6000,
            model_id="ChurnModelForDrivers",
            model_type="churn",
            event="start",
            count=0,
            total=1000,
            user=user,
        )
        session.add(test_start_event)
        session.commit()

        p = Producer(confluent_kafka_config)
        msg_count = 1000
        training_data = generate_n_rows_for_training_data(msg_count, seed=999)
        for i in range(msg_count):
            p.produce(
                f"{test_username}_training_data",
                json.dumps(training_data[i]).encode("utf-8"),
                on_delivery=delivery_report,
            )
        p.flush()

    start = datetime.utcnow()
    while True:
        if datetime.utcnow() - start > timedelta(seconds=10 * 60):
            assert None, "Taking too long to finish while loop. Probably loop is stuck."
        sleep(5)
        with get_session_with_context() as session:
            user = session.exec(
                select(User).where(User.username == test_username)
            ).one()
            event = session.exec(
                select(TrainingStreamStatus)
                .where(TrainingStreamStatus.user == user)
                .where(TrainingStreamStatus.account_id == 6000)
                .order_by(TrainingStreamStatus.id.desc())
                .limit(1)
            ).one()
            logger.info(f"event in test is {event}")
            if event.event == "end":
                display(f"All events for account id {6000}")
                all_events = session.exec(
                    select(TrainingStreamStatus)
                    .where(TrainingStreamStatus.user == user)
                    .where(TrainingStreamStatus.account_id == 6000)
                )
                display([e for e in all_events])
                break


display("starting semaphore")
# with posix_ipc.Semaphore(
#     "/infobip_kafka_topics_semaphore", flags=posix_ipc.O_CREAT, initial_value=1
# ) as sem:
# sem = posix_ipc.Semaphore("/infobip_kafka_topics_semaphore", flags=posix_ipc.O_CREAT)
# sem.acquire(timeout=10 * 60)
display("semaphore started")
create_topics_for_user(username=test_username)
with set_env_variable_context(variable="JOB_EXECUTOR", value="fastapi"):
    with MonkeyPatch.context() as monkeypatch:
        monkeypatch.setattr(
            "__main__.get_count_from_training_data_ch_table",
            lambda account_ids: pd.DataFrame(
                {
                    "curr_count": [999],
                    "AccountId": 6000,
                    "curr_check_on": [datetime.utcnow()],
                }
            ).set_index("AccountId"),
        )
        app, fast_kafka_api_app = create_ws_server(
            assets_path=Path("../assets"), start_process_for_username=None
        )

        @fast_kafka_api_app.run_in_background()
        async def startup_event():
            await process_training_status(
                username=test_username, fast_kafka_api_app=fast_kafka_api_app
            )

        config = uvicorn.Config(app, host="0.0.0.0", port=6010, log_level="debug")

        with run_uvicorn(config):
            # Server started.
            sanitized_print("server started")
            test_process_training_status()

        sanitized_print("server stopped")
        # Server stopped.
# sem.release()
# sem.close()